In [1]:
import urllib

params = 'DRIVER={ODBC Driver 13 for SQL Server};' \
         'SERVER=localhost;' \
         'PORT=1433;' \
         'DATABASE=DbAsftPortal;' \
         'UID=sa;' \
         'PWD=aSj#14776$10036%mM&;'
            
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine

db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

import pandas as pd

sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='DbAsftPortal'
'''

pd.read_sql_query(sql, db)

,TABLE_NAME
0,knox_authtoken
1,tbl_library_book
2,tbl_library_book_author
3,tbl_library_book_publisher
4,tbl_library_book_type
5,tbl_library_loaned_book
6,tbl_meeting_cater_type
7,tbl_meeting_equipment
8,tbl_meeting_request
9,tbl_meeting_room


In [10]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from ldap3 import Server, Connection, ALL, NTLM, SUBTREE
# import ssl, Tls
from django.contrib.auth.models import User
from baseInfo.models import Employee
import re, io
from django.core.files.uploadedfile import InMemoryUploadedFile
 
# try:
LDAP_URL = 'dc.asft.co:389'
username = 'portal'
password = 'Ccontrol'
domain_user = 'asft\\' + username

server = Server(LDAP_URL, get_info=ALL)
conn = Connection(server, user=domain_user, password=password, authentication=NTLM, auto_bind=True)

base = '''dc=asft,dc=co'''
criteria = '''(objectCategory=user)'''
attributes = ['sAMAccountName', 'givenName', 'sn', 'mail', 'userAccountControl', 'thumbnailPhoto']

conn.search(base, criteria, attributes=attributes)
results = conn.response
print(len(results))
for r in results:   
    pattern = re.compile(r'^[a-z|A-Z]+\.[a-z|A-Z|\s]+$')
    if('attributes' in r and pattern.match(str(r['attributes']['sAMAccountName']))):
        username = r['attributes']['sAMAccountName']
        first_name = r['attributes']['givenName']
        last_name = r['attributes']['sn']
        email = r['attributes']['mail']
        thumbnail = r['attributes']['thumbnailPhoto']

        try:
            is_active = 0 if(r['attributes']['userAccountControl'] == 66050 or r['attributes']['userAccountControl'] == 514) else 1
            user, user_created = User.objects.update_or_create(username=username, defaults={'email': email, 'first_name': first_name, 'last_name': last_name, 'is_active': is_active})
#             print('user: ', user, ' ,user_created: ', user_created)
         
            if user_created:
                image_name = 'files/employee_pix/{0}.jpg'.format(username)
                # Set an unusable password -- django-auth-ldap handles this, anyway.  'user_id':user.id, 
                user.set_unusable_password()
                user.save()
                employees = Employee.objects.filter(first_name__exact=first_name, last_name__exact=last_name)
                if(len(employees) == 0):
                    employee = Employee.objects.create(first_name=first_name, last_name=last_name, picture=image_name, email=email, user_id=user.id, gender=False)
                    if(len(thumbnail)>0):
                        buffer = io.BytesIO()
                        buffer.write(thumbnail)
                        image_file = InMemoryUploadedFile(buffer, None, '{0}.jpg'.format(username), 'image/jpg', buffer.getbuffer().nbytes, None)
                        employee.picture.save(image_name, image_file)
        except Exception as e:
            print(e)
            

493
